### Get all statistics and tables from GENESIS, RegioDB, ZensusDB

In [6]:
import pandas as pd
import requests
from dotenv import dotenv_values, load_dotenv

# Load credentials from .env file
load_dotenv()  
USERNAME_GENESIS, PASSWORD_GENESIS, USERNAME_ZENSUS, PASSWORD_ZENSUS, USERNAME_REGIO, PASSWORD_REGIO = dotenv_values().values()

# Database endpoints
BASE_URL_GENESIS = 'https://www-genesis.destatis.de/genesisWS/rest/2020/'
BASE_URL_ZENSUS = 'https://ergebnisse2011.zensus2022.de/api/rest/2020/'
BASE_URL_REGIO = 'https://www.regionalstatistik.de/genesisws/rest/2020/'


In [7]:
def get_stats(username, pw, url, max_pagelength=2500):
    """Get list of statistics from database

    Args: 
        username (str): Username for data base
        pw (str): Password for data base
        url (str): URL of database
        max_pagelength (int): Maximum number of results to return (optional)

    Returns:
        stats_df (Data Frame): Pandas data frame holding list of statistics
        stats_list (list): List of first numbers of EVAS as strings
    """

    response = requests.get(url + 'catalogue/statistics', params={
        'username': username,
        'password': pw,
        'selection': '*',
        'searchcriterion': 'Code',
        'pagelength': max_pagelength,
        'language': 'de',
    })
    response.raise_for_status()
    response.encoding = "UTF-8"

    if response.json()["Status"]["Code"] not in [0, 22, 104]:
        print(response.text)
        raise ValueError("The destatis code suggests that something went wrong. Please check!")
    
    stats_df = pd.DataFrame(response.json()["List"])

    if len(stats_df) >= max_pagelength:
        raise Warning(f"Maximum page length has been reached for index {stat}. Please check!")

    stats_list = stats_df["Code"].str[0].unique().tolist()

    print(f"Success! Found {len(stats_df)} statistics.")

    return stats_df, stats_list


In [8]:
def get_tables(stats_list, username, pw, url, max_pagelength=2500):
    """Get list of tables from database

    Args: 
        stats_list (list): list containing first numbers of statistics (EVAS) for which tables should be returned
        username (str): Username for data base
        pw (str): Password for data base
        url (str): URL of database
        max_pagelength (int): Maximum number of results to return (optional)

    Returns:
        tables (Data Frame): Pandas data frame holding list of tables
    """

    tables = pd.DataFrame()

    for stat in stats_list:

        print(f"Fetching tables for stats index: {stat}/{len(stats_list)} ...\n")

        response = requests.get(url + 'catalogue/tables', params={
        'username': username,
        'password': pw,
        'selection': stat + '*',
        'area': 'all',
        'pagelength': max_pagelength,
        'language': 'de',
        })
        response.raise_for_status()
        response.encoding = "UTF-8"

        if response.json()["Status"]["Code"] not in [0, 22, 104]:
            print(response.text)
            raise ValueError("The destatis code suggests that something went wrong. Please check!")
        
        table_df = pd.DataFrame(response.json()["List"])

        if len(table_df) >= max_pagelength:
            raise Warning(f"Maximum page length has been reached for index {stat}. Please check!")
    
        tables = pd.concat([tables, table_df], axis=0)

    tables.reset_index(drop=True, inplace=True)
    print(f"Success! Found {len(tables)} tables.")

    return tables

##### GENESIS

In [9]:
# Get all statistics
stats_df_genesis, stats_list_genesis = get_stats(USERNAME_GENESIS, PASSWORD_GENESIS, BASE_URL_GENESIS)
stats_df_genesis.to_csv("data/genesis_statistics.csv", index=False)

Success! Found 324 statistics.


In [65]:
# Get all tables
table_df_genesis = get_tables(stats_list_genesis, USERNAME_GENESIS, PASSWORD_GENESIS, BASE_URL_GENESIS)
table_df_genesis.to_csv("data/genesis_tables.csv", index=False)

Fetching tables for stats index: 1/9 ...

Fetching tables for stats index: 2/9 ...

Fetching tables for stats index: 3/9 ...

Fetching tables for stats index: 4/9 ...

Fetching tables for stats index: 5/9 ...

Fetching tables for stats index: 6/9 ...

Fetching tables for stats index: 7/9 ...

Fetching tables for stats index: 8/9 ...

Fetching tables for stats index: 9/9 ...

Success! Found 2689 tables.


##### RegioDB

In [12]:
# Get all statistics
stats_df_regio, stats_list_regio = get_stats(USERNAME_REGIO, PASSWORD_REGIO, BASE_URL_REGIO)
stats_df_regio.to_csv("data/regioDB_statistics.csv", index=False)

Success! Found 101 statistics.


In [13]:
# Get all tables
table_df_regio = get_tables(stats_list_regio, USERNAME_REGIO, PASSWORD_REGIO, BASE_URL_REGIO)
table_df_regio.to_csv("data/regioDB_tables.csv", index=False)

Fetching tables for stats index: 1/9 ...

Fetching tables for stats index: 2/9 ...

Fetching tables for stats index: 3/9 ...

Fetching tables for stats index: 4/9 ...

Fetching tables for stats index: 5/9 ...

Fetching tables for stats index: 6/9 ...

Fetching tables for stats index: 7/9 ...

Fetching tables for stats index: 8/9 ...

Fetching tables for stats index: 9/9 ...

Success! Found 709 tables.


In [22]:
# RegioDB has a strange quirk where all tables relating to the "Regionalatlas" are assigned to the EVAS number
# 99910 but the actual table numbers have a different format (e.g. AI-N-01 or AIG-03-2, they all start with A) 
# Add them here
regionalatlas_df = get_tables(["A"], USERNAME_REGIO, PASSWORD_REGIO, BASE_URL_REGIO)
table_df_regio = pd.concat([table_df_regio, regionalatlas_df], axis=0).reset_index(drop=True)
table_df_regio.to_csv("data/regioDB_tables.csv", index=False)

print(f"Found {len(table_df_regio)} tables in total.")


Fetching tables for stats index: A/1 ...

Success! Found 82 tables.

Found 791 tables in total.


##### Zensus

In [31]:
# Get all statistics
stats_df_zensus, stats_list_zensus = get_stats(USERNAME_ZENSUS, PASSWORD_ZENSUS, BASE_URL_ZENSUS)
stats_df_zensus.to_csv("data/zensusDB_statistics.csv", index=False)

Success! Found 6 statistics.


In [20]:
# Get all tables
table_df_zensus = get_tables(stats_list_zensus, USERNAME_ZENSUS, PASSWORD_ZENSUS, BASE_URL_ZENSUS)
table_df_zensus.to_csv("data/zensusDB_tables.csv", index=False)

Fetching tables for stats index: 1/6 ...

Fetching tables for stats index: 2/6 ...

Fetching tables for stats index: 3/6 ...

Fetching tables for stats index: 4/6 ...

Fetching tables for stats index: 5/6 ...

Fetching tables for stats index: 6/6 ...

Success! Found 754 tables.
